> Proszę zaimplementować funkcję:

    average_score(arr, lowest, highest)
    
> Funkcja ta przyjmuje na wejściu tablicę **arr** liczb rzeczywistych (ich rozkład nie jest znany, ale wszystkie są parami różne) i zwraca średnią wartość podanych liczb, po odrzuceniu **lowest** najmiejszych oraz **highest** największych. Zaimplementowana funkcja powinna być możliwie jak najszybsza. Proszę oszacować jej złożoność czasową (oraz bardzo krótko uzasadnić to oszacowanie).

### Implementacja algorytmu #1 
##### Złożoność: $O(n \cdot log(n))$

Najbardziej trywialna implementacja. Wykorzystujemy jakiś szybki algorytm sortowania liczb (np. Quick Sort), a nastęnie przechodzimy liniowo po posortowanej tablicy, począwszy od indeksu **lowest** (bo usuwamy **lowest** elementów z początku) i skończywszy na **len(arr) - highest - 1** (włącznie) (bo odcinamy **highest** największych).

In [1]:
def quick_sort(arr):
    _quick_sort(arr, 0, len(arr) - 1)
    

def _quick_sort(arr, left_idx, right_idx):
    while left_idx < right_idx:
        pivot_position = _partition(arr, left_idx, right_idx)
        
        if pivot_position - left_idx < right_idx - pivot_position:
            _quick_sort(arr, left_idx, pivot_position)
            left_idx = pivot_position + 1  # I removed a tailing recursion
        else:
            _quick_sort(arr, pivot_position + 1, right_idx)
            right_idx = pivot_position  # I removed a tailing recursion
        

def _partition(arr, left_idx, right_idx):
    pivot = arr[left_idx]
    
    # Partition an array into 2 subarrays of elements lower than or
    # equal to a pivot and of elements greater than a pivot (in this
    # partition algorithm pivot isn't placed on a fixed position but
    # can be also swapped like all the remaining values)
    i = left_idx - 1
    j = right_idx + 1
    while True:
        i += 1
        while arr[i] < pivot: i += 1
            
        j -= 1
        while arr[j] > pivot: j -= 1
        
        if i < j:
            _swap(arr, i, j)
        else:
            return j  # Return a pivot position after the last swap

    
def _swap(arr, i, j):
    arr[i], arr[j] = arr[j], arr[i]
    
    
def average_score(arr, lowest, highest):
    n = len(arr) - highest - lowest
    if n < 0: return None
    if n == 0: return 0
    quick_sort(arr)
    total = 0
    for i in range(lowest, len(arr) - highest):
        total += arr[i]
    return total / (len(arr) - highest - lowest)

###### Kilka testów

In [2]:
import random

# Mają być parami różne, więc używam seta
a = list(set(random.randint(-100, 100) for _ in range(random.randint(0, 25))))
lowest = random.randint(0, len(a) // 2)
highest = random.randint(0, len(a) // 2)
part = sorted(a)[lowest:len(a)-highest]
expected = sum(part) / len(part) if len(part) > 0 else 0
print('Input arr:', a)
print('Sorted arr:', sorted(a))
print('Lowest:', lowest, 'Highest:', highest)
print('Result:', average_score(a, lowest, highest))
print('Expected:', expected)

Input arr: [15, 21, 31, 35, 40, 43, -82, 54, -73, 56, -71, 70, -47, -46, -44, 89, -38, -32, 100, -24, -10, -8]
Sorted arr: [-82, -73, -71, -47, -46, -44, -38, -32, -24, -10, -8, 15, 21, 31, 35, 40, 43, 54, 56, 70, 89, 100]
Lowest: 8 Highest: 0
Result: 36.57142857142857
Expected: 36.57142857142857


### Implementacja algorytmu #2
##### $O(n + k + p + 2 \cdot (n - k) \cdot log(k) + 2 \cdot (n - p) \cdot log(p))  \rightarrow  O(n + n \cdot log(k \cdot p))$

##### Wyjaśnienia

Ten algorytm polega na przechodzeniu przez kolejne wartości w tablicy i ich sumowaniu do pewnej zmiennej, przechowującej całkowitą sumę oraz jednoczesnym korzystaniu z dwóch kopców (Min Heap i Max Heap), do których wrzucamy **lowest** najmniejszych wartości (do kopca Max Heap) i **highest** największych wartości (do kopca Min Heap).

W tym przypadku warto zastosować kopce ze sztywnie zaalokowanymi tablicami (o staym rozmiarze) przechowującymi wartości, ponieważ z góry wiemy, ile dokładnie należy umieścić w nich elementów. W Pythonie zaimitujemy taką tablicę przy pomocy listy, ponieważ w Pythonie nie istnieje domyślnie zaimplementowana statyczna tablica.

In [3]:
class MaxHeap:
    def __init__(self, maxsize):
        if not isinstance(maxsize, int):
            raise TypeError(f"expected 'int', got {str(type(maxsize))[7:-1]}")
        if maxsize <= 0:
            raise ValueError(f"cannot create a {self.__class__.__name__} of a max size {maxsize}")
        self.heap = [None] * maxsize  # Allocate a constant memory space
        self.size = 0
        self._free_idx = 0
    
    def __bool__(self):
        return bool(self._free_idx)
    
    def __len__(self):
        return len(self.heap)
    
    @property
    def heap_size(self):
        return self._free_idx
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def insert(self, val: object):
        if self.heap_size == len(self):
            raise OverflowError(f'insert in a completely filled {self.__class__.__name__}')
        # Add a value as the last node of our Complete Binary Tree
        self.heap[self._free_idx] = val
        # Fix heap in order to satisfy a max-heap property
        self._heapify_up(self.heap_size)
        self._free_idx += 1
        
    def get_max(self) -> object:
        return None if not self.heap else self.heap[0]
        
    def remove_max(self) -> object:
        if self.heap_size == 0:
            raise IndexError(f'remove_max from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self.heap[0]
        # Place the last leaf in the root position
        last_idx = self._free_idx - 1
        last = self.heap[last_idx]
        self.heap[last_idx] = None
        self._free_idx -= 1
        if self.heap_size > 0:
            self.heap[0] = last
            # Fix a heap in order to stisfy a max-heap property
            self._heapify_down(0, self.heap_size)
        return removed
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):  # O(log n)
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            if self.heap[curr_idx] > self.heap[parent_idx]:
                self.swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):  # O(log n)
        # Loop till the current node has a child larger than itself
        # We assume that when we enter a node which both children are
        # smaller than this node, a subtree which a current node is a
        # root of must fulfill a max-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            largest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] > self.heap[curr_idx]: 
                    largest_idx = l
                if r < end_idx and self.heap[r] > self.heap[largest_idx]:
                    largest_idx = r

            if largest_idx != curr_idx:
                self.swap(curr_idx, largest_idx)
                curr_idx = largest_idx
            else:
                break
                
                
class MinHeap:
    def __init__(self, maxsize):
        if not isinstance(maxsize, int):
            raise TypeError(f"expected 'int', got {str(type(maxsize))[7:-1]}")
        if maxsize <= 0:
            raise ValueError(f"cannot create a {self.__class__.__name__} of a max size {maxsize}")
        self.heap = [None] * maxsize  # Allocate a constant memory space
        self.size = 0
        self._free_idx = 0

    def __bool__(self):
        return bool(self._free_idx)

    def __len__(self):
        return len(self.heap)

    @property
    def heap_size(self):
        return self._free_idx

    @staticmethod
    def parent_idx(curr_idx):
        return curr_idx // 2

    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2

    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 1

    def insert(self, val: object):
        if self.heap_size == len(self):
            raise OverflowError(f'insert in a completely filled {self.__class__.__name__}')
        # Add a value as the last node of our Complete Binary Tree
        self.heap[self._free_idx] = val
        # Fix heap in order to satisfy a min-heap property
        self._heapify_up(self.heap_size)
        self._free_idx += 1
        
    def get_min(self) -> object:
        return None if not self.heap else self.heap[0]

    def remove_min(self) -> object:
        if self.heap_size == 0:
            raise IndexError(f'remove_min from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self.heap[0]
        # Place the last leaf in the root position
        last_idx = self._free_idx - 1
        last = self.heap[last_idx]
        self.heap[last_idx] = None
        self._free_idx -= 1
        if self.heap_size > 0:
            self.heap[0] = last
            # Fix a heap in order to stisfy a min-heap property
            self._heapify_down(0, self.heap_size)
        return removed

    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]

    def _heapify_up(self, curr_idx, end_idx=0):  # O(log n)
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            if self.heap[curr_idx] < self.heap[parent_idx]:
                self.swap(curr_idx, parent_idx)
            curr_idx = parent_idx

    def _heapify_down(self, curr_idx, end_idx):  # O(log n)
        # Loop till the current node has a child smaller than itself
        # We assume that when we enter a node which both children are
        # larger than this node, a subtree which a current node is a
        # root of must fulfill a min-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            smallest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] < self.heap[curr_idx]: 
                    smallest_idx = l
                if r < end_idx and self.heap[r] < self.heap[smallest_idx]:
                    smallest_idx = r

            if smallest_idx != curr_idx:
                self.swap(curr_idx, smallest_idx)
                curr_idx = smallest_idx
            else:
                break


def average_score(arr, lowest, highest):
    n = len(arr) - highest - lowest
    if n < 0: return None
    if n == 0: return 0
    
    sum_all = 0
    
    if lowest > 0 and highest > 0:
        min_heap = MinHeap(highest)
        max_heap = MaxHeap(lowest)

        # Store lowest an greatest values in heaps
        for val in arr:
            sum_all += val
            if min_heap.heap_size < len(min_heap):
                min_heap.insert(val)
            elif val > min_heap.get_min():
                min_heap.remove_min()
                min_heap.insert(val)
                
            if max_heap.heap_size < len(max_heap):
                max_heap.insert(val)
            elif val < max_heap.get_max():
                max_heap.remove_max()
                max_heap.insert(val)
                
        # Subtract a sum of values in heaps from the sum_all
        for val in min_heap.heap: sum_all -= val
        for val in max_heap.heap: sum_all -= val
        return sum_all / (len(arr) - lowest - highest)
        
    elif lowest > 0:
        max_heap = MaxHeap(lowest)
        
        # Store lowest an greatest values in heaps
        for val in arr:
            sum_all += val
            
            if max_heap.heap_size < len(max_heap):
                max_heap.insert(val)
            elif val < max_heap.get_max():
                max_heap.remove_max()
                max_heap.insert(val)
                
        # Subtract a sum of values in heaps from the sum_all
        for val in max_heap.heap: sum_all -= val
        return sum_all / (len(arr) - lowest)
    
    elif highest > 0:
        min_heap = MinHeap(highest)

        # Store lowest an greatest values in heaps
        for val in arr:
            sum_all += val
            if min_heap.heap_size < len(min_heap):
                min_heap.insert(val)
            elif val > min_heap.get_min():
                min_heap.remove_min()
                min_heap.insert(val)
                
        # Subtract a sum of values in heaps from the sum_all
        for val in min_heap.heap: sum_all -= val
        return sum_all / (len(arr) - highest)
    
    else:
        for val in arr:
            sum_all += val
        return sum_all / len(arr)

###### Kilka testów

In [4]:
import random

# Mają być parami różne, więc używam seta
a = list(set(random.randint(-100, 100) for _ in range(random.randint(0, 25))))
lowest = random.randint(0, len(a) // 2)
highest = random.randint(0, len(a) // 2)
part = sorted(a)[lowest:len(a)-highest]
expected = sum(part) / len(part) if part else 0
print('Input arr:', a)
print('Sorted arr:', sorted(a))
print('Lowest:', lowest, 'Highest:', highest)
print('Result:', average_score(a, lowest, highest))
print('Expected:', expected)

Input arr: [-96, 64, 97, -61, -26, -56, -55, -54, 76, 13, -50, -16, -13, -72, 26, 88, -100]
Sorted arr: [-100, -96, -72, -61, -56, -55, -54, -50, -26, -16, -13, 13, 26, 64, 76, 88, 97]
Lowest: 3 Highest: 3
Result: -20.727272727272727
Expected: -20.727272727272727


### Implementacja algorytmu #3
##### Złożoność: $O(n + p \cdot log(n) + (n - p) + k \cdot log(n - p) + (n - p - k)) \rightarrow O(n + (p + k) \cdot log(n))$

##### Wyjaśnienia

To rozwiązanie jest podobne do powyższego, ponieważ również opiera się na zastosowaniu kopców. W tym przypadku jednak tworzymy najpierw jeden cały kopiec (Max Heap lub Min Heap) ze wszystkich wartości, odrzucamy z niego odpowiednią liczbę największych/najmniejszych wartości i tworzymy kopiec (Min Heap, gdy poprzedni to był Max Heap lub odwrotnie w przeciwnym wypadku). Z drugiego kopca odrzucamy odpowiednio najmniejsze/największe wartości i liczymy średnią z pozostałych.

##### Oszacowanie złożoności

- $p$ = lowest; $k$ = highest,
- $n$ - budowanie kopca Min Heap z całej tablicy,
- $p \cdot log(n)$ - usuwanie $p$ wartości najmniejszych z kopca,
- $(n - p)$ - budowanie kopca Max Heap z pozostałych $n - p$ elementów,
- $k \cdot log(n - p)$ - usuwanie $k$ wartości największych z kopca,
- $(n - p - k)$ - liniowe sumowanie pozostałych wartości.

In [5]:
class MaxHeap:
    def __init__(self, values=None):
        if values:
            self.heap = list(values) # We make a copy of values in order not to modify them
            self.build_heap()
        else:
            self.heap = []
        
    def __str__(self):  # A 'complete_tree_string' function is required in order to ensure that printing works
        return complete_tree_string(self.heap)
    
    def __bool__(self):
        return bool(self.heap)
    
    @property
    def heap_size(self):
        return len(self.heap)
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def insert(self, val: object):
        # Add a value as the last node of out Complete Binary Tree
        self.heap.append(val)
        # Fix heap in order to satisfy a max-heap property
        self._heapify_up(self.heap_size - 1)
        
    def get_max(self) -> object:
        return None if not self.heap else self.heap[0]
        
    def remove_max(self) -> object:
        if self.heap_size == 0:
            raise IndexError(f'remove_max from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self.heap[0]
        # Place the last leaf in the root position
        last = self.heap.pop()
        if self.heap_size > 0:
            self.heap[0] = last
            # Fix a heap in order to stisfy a max-heap property
            self._heapify_down(0, self.heap_size)
        return removed
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):  # O(log n)
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            if self.heap[curr_idx] > self.heap[parent_idx]:
                self.swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):  # O(log n)
        # Loop till the current node has a child larger than itself
        # We assume that when we enter a node which both children are
        # smaller than this node, a subtree which a current node is a
        # root of must fulfill a max-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            largest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] > self.heap[curr_idx]: 
                    largest_idx = l
                if r < end_idx and self.heap[r] > self.heap[largest_idx]:
                    largest_idx = r

            if largest_idx != curr_idx:
                self.swap(curr_idx, largest_idx)
                curr_idx = largest_idx
            else:
                break
        
    def build_heap(self):   # O (n)
        for i in range(self.heap_size // 2 - 1, -1, -1):
            self._heapify_down(i, self.heap_size)
            
            
class MinHeap:
    def __init__(self, values=None):
        if values:
            self.heap = list(values) # We make a copy of values in order not to modify them
            self.build_heap()
        else:
            self.heap = []
        
    def __str__(self):  # A 'complete_tree_string' function is required in order to ensure that printing works
        return complete_tree_string(self.heap)
    
    def __bool__(self):
        return bool(self.heap)
    
    @property
    def heap_size(self):
        return len(self.heap)
    
    @staticmethod
    def parent_idx(curr_idx):
        return (curr_idx - 1) // 2
    
    @staticmethod
    def left_child_idx(curr_idx):
        return curr_idx * 2 + 1
    
    @staticmethod
    def right_child_idx(curr_idx):
        return curr_idx * 2 + 2
    
    def insert(self, val: object):
        # Add a value as the last node of our Complete Binary Tree
        self.heap.append(val)
        # Fix heap in order to satisfy a min-heap property
        self._heapify_up(self.heap_size - 1)
        
    def get_min(self) -> object:
        return None if not self.heap else self.heap[0]
        
    def remove_min(self) -> object:
        if self.heap_size == 0:
            raise IndexError(f'remove_min from an empty {self.__class__.__name__}')
        # Store a value to be returned
        removed = self.heap[0]
        # Place the last leaf in the root position
        last = self.heap.pop()
        if self.heap_size > 0:
            self.heap[0] = last
            # Fix a heap in order to stisfy a min-heap property
            self._heapify_down(0, self.heap_size)
        return removed
    
    def swap(self, i, j):
        self.heap[i], self.heap[j] = self.heap[j], self.heap[i]
        
    def _heapify_up(self, curr_idx, end_idx=0):  # O(log n)
        while curr_idx > end_idx:
            parent_idx = self.parent_idx(curr_idx)
            if self.heap[curr_idx] < self.heap[parent_idx]:
                self.swap(curr_idx, parent_idx)
            curr_idx = parent_idx
        
    def _heapify_down(self, curr_idx, end_idx):  # O(log n)
        # Loop till the current node has a child smaller than itself
        # We assume that when we enter a node which both children are
        # larger than this node, a subtree which a current node is a
        # root of must fulfill a min-heap property
        while True:
            l = self.left_child_idx(curr_idx)
            r = self.right_child_idx(curr_idx)
            smallest_idx = curr_idx

            if l < end_idx:
                if self.heap[l] < self.heap[curr_idx]: 
                    smallest_idx = l
                if r < end_idx and self.heap[r] < self.heap[smallest_idx]:
                    smallest_idx = r
        
            if smallest_idx != curr_idx:
                self.swap(curr_idx, smallest_idx)
                curr_idx = smallest_idx
            else:
                break
        
    def build_heap(self):   # O (n)
        for i in range(self.heap_size // 2 - 1, -1, -1):
            self._heapify_down(i, self.heap_size)
            
            
def average_score(arr, lowest, highest):
    n = len(arr) - highest - lowest
    if n < 0: return None
    if n == 0: return 0
    if lowest > 0:
        min_heap = MinHeap(arr)
        for _ in range(lowest):
            min_heap.remove_min()
        arr = min_heap.heap
    if highest > 0:
        max_heap = MaxHeap(arr)
        for _ in range(highest):
            max_heap.remove_max()
        arr = max_heap.heap
    # Calculate the average
    return sum(arr) / len(arr)

###### Kilka testów

In [6]:
import random

# Mają być parami różne, więc używam seta
a = list(set(random.randint(-100, 100) for _ in range(random.randint(0, 25))))
lowest = random.randint(0, len(a) // 2)
highest = random.randint(0, len(a) // 2)
part = sorted(a)[lowest:len(a)-highest]
expected = sum(part) / len(part) if part else 0
print('Input arr:', a)
print('Sorted arr:', sorted(a))
print('Lowest:', lowest, 'Highest:', highest)
print('Result:', average_score(a, lowest, highest))
print('Expected:', expected)

Input arr: [25, -13, 61]
Sorted arr: [-13, 25, 61]
Lowest: 1 Highest: 1
Result: 25.0
Expected: 25.0


### Implementacja algorytmu #4 (najlepsza)
##### Złożoność: $O(n)$

Korzystamy z Quick Selecta.

In [7]:
def quick_select(arr, k, left_idx, right_idx):
    pivot_position = _partition(arr, left_idx, right_idx)
        
    if pivot_position > k:
        return quick_select(arr, k, left_idx, pivot_position - 1)
    elif pivot_position < k:
        return quick_select(arr, k, pivot_position + 1, right_idx)
    else:
        return arr[pivot_position]
        
        
def _partition(arr, left_idx, right_idx):
    pivot = arr[right_idx]
    
    # Swap a pivot with the last element
    swap(arr, right_idx, right_idx)
    
    # Partition an array into 2 subarrays of elements lower than or
    # equal to a pivot and of elements greater than a pivot
    i = left_idx
    for j in range(left_idx, right_idx):
        if arr[j] < pivot:
            swap(arr, i, j)
            i += 1
    
    # Place a pivot element on its destination index
    swap(arr, i, right_idx)
    
    return i  # Return a pivot position after the last swap

    
def swap(arr, i, j):
    arr[i], arr[j] = arr[j], arr[i]
    
    
def average_score(arr, lowest, highest):
    n = len(arr) - highest - lowest
    if n < 0: return None
    if n == 0: return 0
    if lowest > 0:
        quick_select(arr, lowest - 1, 0, len(arr) - 1)
    if highest > 0:
        quick_select(arr, len(arr) - highest, lowest, len(arr) - 1)
    # Calculate the average
    sum_ = 0
    for i in range(lowest, len(arr) - highest):
        sum_ += arr[i]
    return sum_ / (len(arr) - highest - lowest)

###### Kilka testów

In [8]:
import random

# Mają być parami różne, więc używam seta
a = list(set(random.randint(-100, 100) for _ in range(random.randint(0, 25))))
lowest = random.randint(0, len(a) // 2)
highest = random.randint(0, len(a) // 2)
part = sorted(a)[lowest:len(a)-highest]
expected = sum(part) / len(part) if part else 0
print('Input arr:', a)
print('Sorted arr:', sorted(a))
print('Lowest:', lowest, 'Highest:', highest)
print('Result:', average_score(a, lowest, highest))
print('Expected:', expected)

Input arr: [-23, -78, 88, -37, 30]
Sorted arr: [-78, -37, -23, 30, 88]
Lowest: 1 Highest: 0
Result: 14.5
Expected: 14.5
